## IMDB Movie Reviews using Word Embeddings in Keras

## [ Please star/upvote if u like it. ]

#### IMPORTING MODULES

In [0]:
import nltk
nltk.download('all',halt_on_error=False)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [0]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
% matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# BeautifulSoup libraray
from bs4 import BeautifulSoup 

import re # regex

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input,Dropout
from keras.models import Model
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


#### LOADING THE DATASET

In [0]:
train = pd.read_csv(r'drive/Colab Notebooks/IMDB Movie Review Kaggle/Keras/labeledTrainData.tsv', header=0,\
                    delimiter="\t", quoting=3)

test = pd.read_csv(r'drive/Colab Notebooks/IMDB Movie Review Kaggle/Keras/testData.tsv',header=0,\
                    delimiter="\t", quoting=3)

In [0]:
print("the train shape is : ",train.shape)
print("the test shape is : ",test.shape)

the train shape is :  (25000, 3)
the test shape is :  (25000, 2)


In [0]:
df=train.copy()

In [0]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [0]:
df.drop('id',axis=1,inplace=True)

In [0]:
df.head()

,sentiment,review
0,1,"""With all this stuff going down at the moment ..."
1,1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,0,"""The film starts with a manager (Nicholas Bell..."
3,0,"""It must be assumed that those who praised thi..."
4,1,"""Superbly trashy and wondrously unpretentious ..."


In [0]:
df['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [0]:
df['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [0]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [0]:
test['review'][0]

'"Naturally in a film who\'s main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there is a craftsmanship and completeness to the film which anyone can enjoy. The pace is steady and constant, the characters full and engaging, the relationships and interactions natural showing that you do not need floods of tears to show emotion, screams to show fear, shouting to show dispute or violence to show anger. Naturally Joyce\'s short story lends the film a ready made structure as perfect as a polished diamond, but the small changes Huston makes such as the inclusion of the poem fit in neatly. It is truly a masterpiece of tact, subtlety and overwhelming beauty."'

In [0]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


#### PRE-PROCESSING & DATA CLEANING

In [0]:
def clean_reviews(review):
    
    # 1. Removing html tags
    review_text = BeautifulSoup(review).get_text()
    
    # 2. Retaining only alphabets.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    
    # 3. Converting to lower case and splitting
    word_tokens= review_text.lower().split()
    
    # 4. Remove stopwords
    stop_words= set(stopwords.words("english"))     
    word_tokens= [w for w in word_tokens if not w in stop_words]
    
    cleaned_review=" ".join(word_tokens)
    return cleaned_review

In [0]:
df['review']=df['review'].apply(clean_reviews)
test['review']=test['review'].apply(clean_reviews)

In [0]:
print("the train shape is : ",df.shape)
print("the test shape is : ",test.shape)

the train shape is :  (25000, 2)
the test shape is :  (25000, 2)


In [0]:
df['review'][0]

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

In [0]:
test['review'][0]

'naturally film main themes mortality nostalgia loss innocence perhaps surprising rated highly older viewers younger ones however craftsmanship completeness film anyone enjoy pace steady constant characters full engaging relationships interactions natural showing need floods tears show emotion screams show fear shouting show dispute violence show anger naturally joyce short story lends film ready made structure perfect polished diamond small changes huston makes inclusion poem fit neatly truly masterpiece tact subtlety overwhelming beauty'

#### ENCODING 

In [0]:
'''

need to find no of unique words. this will be taken by the keras embedding layer as 'input_dim' parameter.

'''
master=[]
for e in df['review']:
  master.append(e)

words=[]
for review in master:
  tok=review.split()
  for r in tok:
    words.append(r)
    
master=[]
for e in test['review']:
  master.append(e)
  
for review in master:
  tok=review.split()
  for r in tok:
    words.append(r)
    
print(len(set(words)))
  
vocab_size=len(set(words))


101245


In [0]:
tokenizer=Tokenizer(num_words=vocab_size)   # see the other notebook named 'keras tokenizer'
a=tokenizer.fit_on_texts(list(df['review']))
encoded_train=tokenizer.texts_to_sequences(list(df['review']))

tokenizer=Tokenizer(num_words=vocab_size)
a=tokenizer.fit_on_texts(list(test['review']))
encoded_test=tokenizer.texts_to_sequences(list(test['review']))

print()


In [0]:
'''

need to find the maximum no of words in any document. 

we will pad the shorter documents with zeros. 

all the document should be of same length. this will be required by the Embedding layer in keras.

the length of maximum document will then be the length of all the documents after padding the shorter ones 

and will be taken by the 'input_length' parameter in the Embedding layer.

'''

maxi=-1
for str in df['review']:
  tokens=nltk.word_tokenize(str)
  if(maxi<len((tokens))):
    maxi=len(tokens)
    
for str in test['review']:
  tokenss=nltk.word_tokenize(str)
  if(maxi<len((tokens))):
    maxi=len(tokens)
    
print(maxi)

1416


In [0]:
# therefore we need to encode each document to consisting of 1416 words
max_doc_len=maxi
pad_encoded_train=pad_sequences(encoded_train,max_doc_len)
pad_encoded_test=pad_sequences(encoded_test,max_doc_len)

Y=df['sentiment'].values
Y=to_categorical(Y,2)

print(pad_encoded_train.shape)

(25000, 1416)


#### CREATING WORD EMBEDDINGS and TRAINING USING NN

In [0]:
vocab_size=vocab_size
embed_size=256   # paramter to deal with. if chosen 300 then atak raha hai. try:(
max_doc_len=maxi

In [0]:
word_input=Input(shape=(max_doc_len,),dtype='float64')  # shape of input. each document has 1416 element or words
word_embedding=Embedding(input_dim=vocab_size,output_dim=embed_size,input_length=max_doc_len)(word_input) # creating the embedding
word_vec=Flatten()(word_embedding) # flatten
word_vec=Dense(32,activation='relu')(word_vec)
word_vec=Dense(2,activation='softmax')(word_vec)  # binary classification
embed_model =Model([word_input],word_vec) # combining all into a Keras model

In [0]:
# compiling the model. parameters can be tuned as always.
embed_model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),loss='binary_crossentropy',metrics=['acc'])

In [0]:
embed_model.summary() # summary of the model.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1416)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1416, 256)         25918720  
_________________________________________________________________
flatten_1 (Flatten)          (None, 362496)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                11599904  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 37,518,690
Trainable params: 37,518,690
Non-trainable params: 0
_________________________________________________________________


In [1]:
batch_size=128   # neuran net training params
epochs=5
# splitting into train and val sets.
x_train,x_test,y_train,y_test=train_test_split(pad_encoded_train,Y,test_size=0.20,random_state=42) 

In [0]:
embed_model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test,y_test)) # fitting on train and validating on val sets.

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 14s 707us/step - loss: 0.7271 - acc: 0.7402 - val_loss: 0.2762 - val_acc: 0.8844
Epoch 2/5
20000/20000 [==============================] - 12s 614us/step - loss: 0.0875 - acc: 0.9738 - val_loss: 0.2773 - val_acc: 0.8868
Epoch 3/5
20000/20000 [==============================] - 12s 616us/step - loss: 0.0120 - acc: 0.9995 - val_loss: 0.2903 - val_acc: 0.8894
Epoch 4/5
20000/20000 [==============================] - 12s 618us/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.3034 - val_acc: 0.8886
Epoch 5/5
20000/20000 [==============================] - 12s 616us/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.3147 - val_acc: 0.8904


#### SAVING FOR PREDICTIONS on KAGGLE

In [0]:
pred=embed_model.predict(pad_encoded_test)
pred=np.argmax(pred,axis=1)
d={'id':test['id'],'sentiment':pred}
answer=pd.DataFrame(d)
answer.to_csv('answer.csv',index=False)
from google.colab import files
files.download('answer.csv')

## THE END!!!

## [ please star/upvote if u liked it. ]